In [1]:
using BenchmarkTools
using NPZ
using JSON

Compile it with ifort, and check the exported symbols:
```
call "c:\Program Files (x86)\IntelSWTools\compilers_and_libraries_2019.5.281\windows\bin\ifortvars.bat" intel64
ifort -O2 /dll clipping.f90 -o ifort_clipping.dll
DUMPBIN /exports clipping.dll
```

If not on Windows, maybe add `-static-intel`, i.e.:
```
ifort -O2 /dll -static-intel clipping.f90 -o ifort_clipping.dll
```

Install MinGW for gfortran, make sure to link libgfortran statically or the DLL will not be accepted.
```
gfortran -O2 -shared -static-libgfortran clipping.f90 -o gfortran_clipping.dll
```

In [2]:
function ifort_area_of_intersection(a, b)
    ndim, nvertex, ntriangles = size(a)
    out = Vector{Float64}(undef, ntriangles)
    ccall(
        (:area_of_intersection, "./src/fortran/ifort_clipping.dll"),
        Nothing,
        (Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}),
        Ref(ndim), Ref(nvertex), Ref(ntriangles), a, b, out,
    )
    return out
end

function gfortran_area_of_intersection(a, b)
    ndim, nvertex, ntriangles = size(a)
    out = Vector{Float64}(undef, ntriangles)
    ccall(
        (:area_of_intersection, "./src/fortran/gfortran_clipping.dll"),
        Nothing,
        (Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}),
        Ref(ndim), Ref(nvertex), Ref(ntriangles), a, b, out,
    )
    return out
end

gfortran_area_of_intersection (generic function with 1 method)

In [3]:
a = permutedims(npzread("triangles_a.npy"), (3, 2, 1))
b = permutedims(npzread("triangles_b.npy"), (3, 2, 1))

2×3×10000000 Array{Float64,3}:
[:, :, 1] =
 0.569679   0.593738   0.447445  
 0.0357082  0.0752075  0.00638897

[:, :, 2] =
 0.422911  0.871233  0.625924
 0.573643  0.930234  0.810233

[:, :, 3] =
 0.91546   0.185819  0.695099
 0.227005  0.354787  0.227898

...

[:, :, 9999998] =
 0.0961949  0.171892  0.414149
 0.191146   0.15893   0.388838

[:, :, 9999999] =
 0.438089  0.643506  0.106764
 0.280221  0.529563  0.763313

[:, :, 10000000] =
 0.766416  0.0420939  0.50417 
 0.59956   0.849241   0.471104

In [4]:
times = Dict()
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
    aa = a[:, :, 1:s]
    bb = b[:, :, 1:s]
    res = @belapsed(ifort_area_of_intersection($aa, $bb))
    times[s] = res
end

open("timing/julia-ifort.json","w") do f
    JSON.print(f, times)
end

npzwrite("answers/julia-ifort.npy", ifort_area_of_intersection(a[:, :, 1:10_000], b[:, :, 1:10_000]))

In [5]:
times = Dict()
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
    aa = a[:, :, 1:s]
    bb = b[:, :, 1:s]
    res = @belapsed(gfortran_area_of_intersection($aa, $bb))
    times[s] = res
end

open("timing/julia-gfortran.json","w") do f
    JSON.print(f, times)
end

npzwrite("answers/julia-gfortran.npy", gfortran_area_of_intersection(a[:, :, 1:10_000], b[:, :, 1:10_000]))